<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/CNN_OverSamp_ROCAUC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "oversample"
MODEL_NAME = "oversample"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 15
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 71kB 4.6MB/s 
  Found existing installation: fsspec 0.5.2
    Uninstalling fsspec-0.5.2:
      Successfully uninstalled fsspec-0.5.2


In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(3962284, 8)

In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [9]:
train_df.op_gender.value_counts()

M    2900753
W    1061531
Name: op_gender, dtype: int64

In [10]:
dev_df.op_gender.value_counts()

M    1948371
W     344536
Name: op_gender, dtype: int64

In [0]:
male = train_df[train_df.op_gender=='M']
female = train_df[train_df.op_gender=='W']

In [0]:
from sklearn.utils import resample

female_upsampled = resample(female,
                          replace=True, # sample with replacement
                          n_samples=len(male), # match number in majority class
                          random_state=27) # reproducible results


In [0]:
upsampled_train = pd.concat([male,female_upsampled])

In [14]:
upsampled_train.op_gender.value_counts()

W    2900753
M    2900753
Name: op_gender, dtype: int64

In [0]:
y_train = upsampled_train.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(upsampled_train.response_text.values)
new_sentences_test = get_text_list(dev_df.response_text.values)

In [19]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 07 minutes, 00 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [22]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
\ [4 files][  2.1 GiB/  2.1 GiB]   85.3 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
/
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

In [25]:
embedding_matrix.shape

(590946, 100)

In [26]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           59094600  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
# smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
# smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
# # Set up oversampling with SMOTE (Synthetic Minority Oversampling Technique)
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=12, ratio = 1.0) # Will bring final ratio to 1.0
# x_train_res, y_train_res = sm.fit_sample(X_train, y_train)


In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [30]:
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 5801506 samples, validate on 2000000 samples
Epoch 1/15
5801506/5801506 [==============================] - 152s 26us/sample - loss: 0.6088 - acc: 0.6300 - val_loss: 0.6300 - val_acc: 0.6483
Epoch 2/15
5801506/5801506 [==============================] - 156s 27us/sample - loss: 0.5907 - acc: 0.6460 - val_loss: 0.6264 - val_acc: 0.6541
Epoch 3/15
5801506/5801506 [==============================] - 153s 26us/sample - loss: 0.5841 - acc: 0.6523 - val_loss: 0.6310 - val_acc: 0.6484
Epoch 4/15
5801506/5801506 [==============================] - 154s 26us/sample - loss: 0.5799 - acc: 0.6564 - val_loss: 0.6075 - val_acc: 0.6803
Epoch 5/15
5801506/5801506 [==============================] - 152s 26us/sample - loss: 0.5768 - acc: 0.6595 - val_loss: 0.6176 - val_acc: 0.6643
Epoch 6/15
5801506/5801506 [==============================] - 153s 26us/sample - loss: 0.5745 - acc: 0.6618 - val_loss: 0.6340 - val_acc: 0.6350
Epoch 7/15
5801506/5801506 [==============================] - 153s 26us/samp

In [31]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.6746
Testing Accuracy:  0.6260


In [32]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ROC AUC: 0.6154624515067939


In [0]:
preds = model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [38]:
wrong_preds.op_gender.value_counts()

M    689618
W    167946
Name: op_gender, dtype: int64

In [39]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.37777720596605097

In [40]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1142204,5858762,12888814,M,194526,Mirelle,thanks!,Don Beyer,Congress_Democratic,0.461200,W
70350,2188265,38794897,M,37611,Debbie,as always mr. brown is there for the people.,Sherrod Brown,Congress_Democratic,0.346689,W
421831,3242170,18430916,M,99378,Samantha,i admired you since my teens dear sr! thanks for all these years helping the cuban people,Bob Menendez,Congress_Democratic,0.361926,W
702008,4951062,46506154,M,160274,Barbara,good luck at the debates!,Marco Rubio,Congress_Republican,0.362442,W
1269008,7592822,54387439,M,291539,Terry,"my, how politically correct seals have become. we wouldnt want the best person for the job , now...",Ryan Zinke,Congress_Republican,0.265777,W
651195,4900249,46506154,M,160223,Joe,"look who is the con-artist, rubio, a guy from old cuban descendent. if you bring a turkey fom c...",Marco Rubio,Congress_Republican,0.214419,W
136223,2292081,89989833,M,49092,Brownie,you guys get off it and start demanding action! please! this is ridiculous.,Ted Poe,Congress_Republican,0.443442,W
780163,5029217,46506154,M,160592,William,do you listen to the critics? address those concerns? the mudslinging? it will only get more int...,Marco Rubio,Congress_Republican,0.401647,W
1329507,7747801,99469231,M,296108,Teresa,"finally thank you for speaking up even if it is not popular with your party. you are a ""hero"" wh...",John McCain,Congress_Republican,0.379763,W
679458,4928512,46506154,M,160235,Shay,you are a worthless lying establishment fraud.,Marco Rubio,Congress_Republican,0.311604,W


In [41]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1981700,11031092,86569077,W,389862,Dana,"we could use this theory in bloomgburg, ny!!! local government corruption & violations against a...",Kirsten Gillibrand,Congress_Democratic,0.999999,M
1981701,11031093,86569077,W,389862,Dana,"we could use this theory in bloomgburg, ny!!! local government corruption & violations against a...",Kirsten Gillibrand,Congress_Democratic,0.999999,M
387238,3137329,23984618,W,83488,Raymond,http://crooksandliars.com/2014/05/rep-mckeon-benghazi-evidence-contradicts [[share]],Ann Wagner,Congress_Republican,0.999999,M
2043255,11092647,86569077,W,390557,Ernest,@craig ramage - just because some republican judge calls it inconstitutional it doesnt mean it i...,Kirsten Gillibrand,Congress_Democratic,0.999998,M
1995885,11045277,86569077,W,390010,James,"never give up, never surrender! gillibrand 2016",Kirsten Gillibrand,Congress_Democratic,0.999988,M
2123572,11264552,77234050,W,399731,Carol,http://samuel-warde.com/2013/10/house-gop-busted-senator-van-hollen-conveniently-changing-law-vi...,Cathy McMorris Rodgers,Congress_Republican,0.999983,M
427436,3360671,76269753,W,103742,Виктор,http://www.bohemian.com/northbay/senator-warbucks/content?oid=2181485 [[share]],Dianne Feinstein,Congress_Democratic,0.999974,M
429155,3362390,76269753,W,103745,Виктор,http://www.bohemian.com/northbay/senator-warbucks/content?oid=2181485 [[share]],Dianne Feinstein,Congress_Democratic,0.999974,M
1930513,10979905,86569077,W,389357,Jeremy,"thank you senator gillibrand. just called my senator, durbin, to ask him to support your filibus...",Kirsten Gillibrand,Congress_Democratic,0.999952,M
2051378,11100770,86569077,W,390722,John,franken / gillibrand 2016,Kirsten Gillibrand,Congress_Democratic,0.999951,M


In [42]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
90369,2208284,38794897,M,38359,Mb,"well, steve, do not you know that when people complain about congress, they ususally are not com...",Sherrod Brown,Congress_Democratic,0.000009,W
991634,5240688,46506154,M,161317,Laura,"keep the faith ms. ayotte, you are good for new hampshire and our country !! you have more supp...",Marco Rubio,Congress_Republican,0.000019,W
887546,5136600,46506154,M,160986,Kim,good luck joni...................jan,Marco Rubio,Congress_Republican,0.000059,W
889894,5138948,46506154,M,161005,Sven,please send jeanne shaheen back to minnesota.,Marco Rubio,Congress_Republican,0.000060,W
1160577,6216173,20416792,M,222328,Sandy,is this the one co sponsored with rep. mia b love ? thank you,Trent Kelly,Congress_Republican,0.000069,W
1132701,5381755,46506154,M,162152,Laura,congressional candidate frank guinta did this in nh!,Marco Rubio,Congress_Republican,0.000094,W
2180513,11321493,95515852,M,402017,Muha,love u jan,Bill Huizenga,Congress_Republican,0.000104,W
900111,5149165,46506154,M,161080,Teresa,patty murray has declared a war on women and religion. shame on her!!,Marco Rubio,Congress_Republican,0.000115,W
709620,4958674,46506154,M,160284,David,puppeteers of the unsustainable establishment need to retire from politricks and stop making dec...,Marco Rubio,Congress_Republican,0.000116,W
2204222,11908399,17310,M,440152,Carlos,http://petitions.moveon.org/sign/senator-stabenow-support,Sander Levin,Congress_Democratic,0.000121,W
